In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
from mlp import MLP, Layer, InputLayer, OutputLayer, MultinoulliML
from utils import Adam

In [ ]:
def epoch_it(X, epochs, batchsize):
    n, _ = X.shape
    N = int(n/batchsize)
    for _ in range(epochs):
        for i in range(N):
            yield X[i*batchsize : (i+1)*batchsize]

def file_it(path, skiplines=0):
    with open(path) as f:
        for _ in range(skiplines):
            f.readline()
        yield from f

In [ ]:
network = MLP(
    [
        InputLayer(28*28),
        Layer(100),
        Layer(16),
        Layer(16),
        Layer(16),
        OutputLayer(10)
    ],
    model=MultinoulliML(),
    optimizer=Adam(eps=0.005, clip_threshold=None)
)

In [ ]:
batchsize = 20
epochs = 40

data = np.loadtxt("../data/train.csv", skiprows=1, delimiter=",", dtype=int)
valset = data[:100]
valset = (valset[:, 1:] / 255, valset[:, 0])
trainset = data[100:]
it = epoch_it(trainset, epochs, batchsize=batchsize)

network.init_training(batchsize=batchsize, validation_set=valset)

In [ ]:
for batch in it:
    X, Y = (batch[:,1:]/255, batch[:,0])
    cost, accuracy = network.train_minibatch((X, Y))
    print(accuracy, cost)

In [ ]:
network.cleanup_training()

In [ ]:
it = map(
    lambda line: np.fromstring(line, sep=",", dtype=np.uint8),
    file_it("../data/test.csv", skiplines=1),
)

In [ ]:
data = next(it)
n = network.feedforward(data/255)
print(n)
plt.imshow(data.reshape((28,28)), cmap="gray", vmin = 0, vmax = 255)

In [ ]:
network.output_layer.A

In [ ]:
class ParentA:
    def func(self, a):
        print("parent a", a)

class ParentB:
    def func(self, a):
        print("parent b", a)

class Derived(ParentA, ParentB):
    def func(self, a):
        super().func(a=a)
        print("derived", a)

In [ ]:
Derived().func("hello")

In [ ]:
import numpy as np
x = None
np.multiply(3, np.eye(3,3), out = x)

In [ ]:
x = 0 + np.inf

In [ ]:
x = 9
np.sqrt(x, out=x)

In [ ]:
(_, _, _, b) = (1,2,3,4)

In [ ]:
np.argmax(np.eye(3,3), axis=-1)

In [ ]:
x = np.array([1,2,3])
y = np.array([3,2,1])

np.average(x==y)

In [ ]:
import numpy as np 
x=np.eye(3,3)

In [ ]:
pos = (0,0)
x[*pos]

In [ ]:
int(2.6)

In [ ]:
x = True
if x:
    print("foo")
elif x:
    print("bar")

In [ ]:
str(1.0)

In [ ]:
import numpy as np
np.identity(3).reshape(-1).base

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
def plot_examples(colormaps):
    """
    Helper function to plot data with associated colormap.
    """
    np.random.seed(19680801)
    data = np.random.randn(30, 30)
    n = len(colormaps)
    fig, axs = plt.subplots(1, n, figsize=(n * 2 + 2, 3),
                            layout='constrained', squeeze=False)
    for [ax, cmap] in zip(axs.flat, colormaps):
        psm = ax.pcolormesh(data, cmap=cmap, rasterized=True, vmin=-4, vmax=4)
        fig.colorbar(psm, ax=ax)
    plt.show()

In [ ]:
colors = ["#e6550d","#fdd0a2","#000000","#c7e9c0","#31a354"]
nodes = [0.0, 0.49, 0.5, 0.51, 1.0]
cmap = LinearSegmentedColormap.from_list("mycmap", list(zip(nodes, colors)))
plot_examples([cmap])

In [ ]:
cmap([0.1,0.2])

In [ ]:
x = np.identity(4)
np.concatenate((x[:1],x[-1:]))